In [0]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

In [0]:
dbutils.notebook.help()

The notebook module.
 exit(value: String): void -> This method lets you exit a notebook with a value run(path: String, timeoutSeconds: int, arguments: Map): String -> This method runs a notebook and returns its exit value

In [0]:
# dbutils.notebook.run('Worker Notebook',60)  
# Will fail in Databricks Community

In [0]:
# dbutils.notebook.run('/Users/jason.kryske@gmail.com/Delta Tables/Delta Table Time Travel',60) 
#  Will Fail in Databricks Community

In [0]:
try:
    dbutils.notebook.run('Worker Notebook',60) # Since errors stop the notebook, you can use Python's native error handling
except:
    print('Could not run notebook in Databricks community')

Could not run notebook in Databricks community


In [0]:
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getAll: Map -> Retrieves a mapping of all current values of the input widgets getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
dbutils.widgets.text('input_widget','','provide an input')

In [0]:
dbutils.widgets.get('input_widget')

'hello'

In [0]:
try:
    dbutils.notebook.run('Worker Notebook',60, {'intput_widget':'From Master Notebook','argument2':'second string'}) #Updates the widget created in previous cells
    print('widget updated')
except:
    print('Could not run notebook in Databricks community')

Could not run notebook in Databricks community


In [0]:
orders = (spark.read.table('orders_db.orders_bronze'))
orders.limit(5).display()

id,created_at,user_id,product_id,quantity,unit_price
2059,2017-07-20T21:15:35.567Z,286,120,46,55.670000
1857,2018-04-10T21:26:19.545Z,261,147,23,44.430000
1769,2017-03-04T18:05:05.784Z,246,155,13,29.180000
1685,2018-08-25T10:35:43.551Z,236,52,10,69.120000
1638,2018-06-07T14:00:00.269Z,233,136,58,70.140000


In [0]:
def total_order_amount(price, quantity):
    return price * quantity

In [0]:
orders_with_total = orders.withColumn('order_total',total_order_amount(orders.unit_price, orders.quantity))

In [0]:
orders_with_total.limit(5).display()

id,created_at,user_id,product_id,quantity,unit_price,order_total
2059,2017-07-20T21:15:35.567Z,286,120,46,55.670000,2560.820000
1857,2018-04-10T21:26:19.545Z,261,147,23,44.430000,1021.890000
1769,2017-03-04T18:05:05.784Z,246,155,13,29.180000,379.340000
1685,2018-08-25T10:35:43.551Z,236,52,10,69.120000,691.200000
1638,2018-06-07T14:00:00.269Z,233,136,58,70.140000,4068.120000


In [0]:
%run "/Users/jason.kryske@gmail.com/Modularize code and Notebooks/orders_module" 



In [0]:
# Use at the start of notebook so all functions are immediately usable


Can use ./orders_module as the path because it is in the same folder. <br>
Can also use ../orders_module as the relative path from the parent folder.


In [0]:
orders_with_total_2 = orders.withColumn('order_total',total_order_amount_from_other_nb(orders.unit_price, orders.quantity))

In [0]:
orders_with_total_2.limit(5).display()

id,created_at,user_id,product_id,quantity,unit_price,order_total
2059,2017-07-20T21:15:35.567Z,286,120,46,55.670000,2560.820000
1857,2018-04-10T21:26:19.545Z,261,147,23,44.430000,1021.890000
1769,2017-03-04T18:05:05.784Z,246,155,13,29.180000,379.340000
1685,2018-08-25T10:35:43.551Z,236,52,10,69.120000,691.200000
1638,2018-06-07T14:00:00.269Z,233,136,58,70.140000,4068.120000


In [0]:
orders.createTempView('orders_sql_view')

In [0]:
%sql
select * from orders_sql_view
limit 5

id,created_at,user_id,product_id,quantity,unit_price
2059,2017-07-20T21:15:35.567Z,286,120,46,55.670000
1857,2018-04-10T21:26:19.545Z,261,147,23,44.430000
1769,2017-03-04T18:05:05.784Z,246,155,13,29.180000
1685,2018-08-25T10:35:43.551Z,236,52,10,69.120000
1638,2018-06-07T14:00:00.269Z,233,136,58,70.140000


**Using Python Functions on SQL table views**

In [0]:
spark.udf.register('total_order_amount_from_other_nb',total_order_amount_from_other_nb) # first argumentwill be the functions name (I'm keeping the same one)/ The second is the function

<function __main__.total_order_amount_from_other_nb(price, quantity)>

In [0]:
%sql
select 
  *,
  round(total_order_amount_from_other_nb(unit_price,quantity),2) as order_total_sql
from 
  orders_sql_view
limit 5

id,created_at,user_id,product_id,quantity,unit_price,order_total_sql
2059,2017-07-20T21:15:35.567Z,286,120,46,55.670000,2560.82
1857,2018-04-10T21:26:19.545Z,261,147,23,44.430000,1021.89
1769,2017-03-04T18:05:05.784Z,246,155,13,29.180000,379.34
1685,2018-08-25T10:35:43.551Z,236,52,10,69.120000,691.2
1638,2018-06-07T14:00:00.269Z,233,136,58,70.140000,4068.12
